In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 446 ms (started: 2021-08-28 17:50:06 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #3 Aug 28, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main 0c6ecfd] 更新 #3 Aug 28, 2021
 4 files changed, 1609 insertions(+), 2 deletions(-)
 create mode 100644 "\345\212\240\345\277\253\346\250\241\345\236\213\350\256\255\347\273\203.ipynb"
 create mode 100644 "\346\240\274\345\274\217\346\214\207\345\215\227.ipynb"
 create mode 100644 "\347\256\241\347\220\206\346\225\260\346\215\256.ipynb"


To git@github.com:ustchope/pytorch_lightning_study.git
   c5f194e..0c6ecfd  main -> main


time: 3.97 s (started: 2021-08-28 17:50:15 +08:00)


Lightning 的一个主要目标是提高可读性和可重复性。 想象一下，查看任何 GitHub 存储库，找到一个闪电模块，并确切地知道去哪里寻找你关心的东西。

本风格指南的目标是鼓励闪电代码的结构相似。

In [6]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 4.1 s (started: 2021-08-28 20:24:02 +08:00)


# LightningModule

这些是构建 LightningModule 的最佳实践

## 系统与模型

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtwrs8jqqkj61620ouwft02.jpg)

LightningModule 背后的主要原则是一个完整的系统应该是自包含的。 在 Lightning 中，我们区分系统和模型。

模型类似于 resnet18、RNN 等。

系统定义了模型集合如何相互交互。 这方面的例子是：
* GANs
* Seq2Seq
* BERT
* etc

LightningModule 可以定义系统和模型。

这是一个定义模型的 LightningModule：

In [7]:
class LitModel(pl.LightningModule):
    def __init__(self, num_layers: int = 3):
        super().__init__()
        self.layer_1 = nn.Linear()
        self.layer_2 = nn.Linear()
        self.layer_3 = nn.Linear()

time: 751 µs (started: 2021-08-28 20:24:09 +08:00)


这是一个定义系统的 LightningModule：

In [ ]:
class LitModel(LightningModule):
    def __init__(self, encoder: nn.Module = None, decoder: nn.Module = None):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

对于快速原型设计，在 LightningModule 中定义所有计算通常很有用。 为了可重用性和可扩展性，最好传入相关的主干。

## 独立的

LightningModules应该是独立的。 一个很好的测试来看看你的模型是如何独立的，是问自己这个问题：

“有人可以在不了解内部结构的情况下将此文件放入 Trainer 吗？”

例如，我们将优化器与模型结合使用，因为大多数模型需要具有特定学习率调度器的特定优化器才能正常工作。

## Init

LightningModules 倾向于停止自包含的第一个地方是在 init 中。 尝试在 init 中定义所有相关的合理默认值，以便用户不必猜测。

这是一个示例，其中用户必须搜索文件以找出如何初始化此 LightningModule。

In [ ]:
class LitModel(LightningModule):
    def __init__(self, params):
        self.lr = params.lr
        self.coef_x = params.coef_x

这样定义的模型给您留下了许多问题； coef_x 是什么？ 它是一个字符串吗？ 浮动？ 范围是多少？ 等等…

相反，在你的初始化中明确

In [ ]:
class LitModel(LightningModule):
    def __init__(self, encoder: nn.Module, coeff_x: float = 0.2, lr: float = 1e-3):
        ...

现在用户不必猜测了。 相反，他们知道值类型，并且模型有一个合理的默认值，用户可以立即看到值。

## 方法的顺序

`LightningModule` 中唯一需要的方法是：
* `init`
* `training_step`
* `configure_optimizers`

但是，如果您决定实现其余的可选方法，建议的顺序是：
* 模型/系统定义 (`init`)
* 如果进行推理，则定义`forward`
* 训练挂钩
* 验证钩子
* 测试钩子
* `configure_optimizers`
* 任何其他钩子

在实践中，这段代码看起来像：

In [ ]:
class LitModel(pl.LightningModule):

    def __init__(...):

    def forward(...):

    def training_step(...):

    def training_step_end(...):

    def training_epoch_end(...):

    def validation_step(...):

    def validation_step_end(...):

    def validation_epoch_end(...):

    def test_step(...):

    def test_step_end(...):

    def test_epoch_end(...):

    def configure_optimizers(...):

    def any_extra_hook(...):

## Forward vs training_step

我们建议使用前向进行推理/预测并保持 training_step 独立

In [ ]:
def forward(self, x):
    embeddings = self.encoder(x)


def training_step(self):
    x, y = ...
    z = self.encoder(x)
    pred = self.decoder(z)
    ...

但是在使用DataParallel时，需要手动调用forward

In [ ]:
def training_step(self):
    x, y = ...
    z = self(x)  # < ---------- instead of self.encoder(x)
    pred = self.decoder(z)
    ...

# 数据

这些是处理数据的最佳实践。

## 数据加载器

Lightning 使用数据加载器来处理通过系统的所有数据流。 每当您构建数据加载器时，请确保调整工作人员的数量以获得最大效率。

> 确保不要将 ddp_spawn 与 num_workers > 0 一起使用，否则您的代码会出现瓶颈。

## DataModules

Lightning 引入了DataModules。 数据加载器的问题在于共享完整数据集通常仍然具有挑战性，因为所有这些问题都需要回答：
* 使用了哪些拆分？
* 这个数据集有多少个样本？
* 使用了哪些转换？
* 等等…

正是出于这个原因，我们建议您使用数据模块。 这在协作时特别重要，因为它也会为您的团队节省大量时间。

他们所需要做的就是将数据模块放入闪电训练器中，而不必担心对数据做了什么。

这对于学术和企业环境都是如此，在这些环境中，数据清理和临时指令会减慢想法迭代的进度。